In [12]:
# Use %pip for better session persistence within the kernel
%pip install ultralytics -q

import os
import cv2
import numpy as np
import yaml
from tqdm import tqdm
from ultralytics import YOLO

# Verify Dataset Paths
ROOT_DIR = '/kaggle/input/semantic-drone-dataset/dataset/semantic_drone_dataset'
IMAGE_PATH = os.path.join(ROOT_DIR, 'original_images')
MASK_PATH = os.path.join(ROOT_DIR, 'label_images_semantic')

if os.path.exists(IMAGE_PATH):
    print(f"✅ Found {len(os.listdir(IMAGE_PATH))} Images and {len(os.listdir(MASK_PATH))} Masks.")
else:
    print("❌ ERROR: Data not found. Please click '+ Add Input' and search for 'Aerial Semantic Segmentation Drone Dataset'.")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 16.8 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
✅ Found 400 Images and 400 Masks.


In [13]:
LABEL_DIR = '/kaggle/working/labels'
os.makedirs(LABEL_DIR, exist_ok=True)

# Plantable IDs: 2 (dirt), 3 (grass), 4 (gravel)
PLANTABLE_IDS = [2, 3, 4]

print("Converting masks to binary YOLO format...")
for mask_name in tqdm(os.listdir(MASK_PATH)):
    if not mask_name.endswith('.png'): continue
    
    mask = cv2.imread(os.path.join(MASK_PATH, mask_name), 0)
    h, w = mask.shape
    
    with open(os.path.join(LABEL_DIR, mask_name.replace('.png', '.txt')), 'w') as f:
        for class_id in range(0, 24):
            category = 0 if class_id in PLANTABLE_IDS else 1
            binary_mask = np.where(mask == class_id, 255, 0).astype(np.uint8)
            contours, _ = cv2.findContours(binary_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            
            for cnt in contours:
                if len(cnt) < 5: continue 
                poly = cnt.reshape(-1, 2).astype(np.float32)
                poly[:, 0] /= w
                poly[:, 1] /= h
                line = f"{category} " + " ".join([f"{x:.6f} {y:.6f}" for x, y in poly])
                f.write(line + "\n")

Converting masks to binary YOLO format...


100%|██████████| 400/400 [15:35<00:00,  2.34s/it]


In [15]:
import shutil

# 1. Create linked dataset structure (fixes 'No labels found' error)
os.makedirs('/kaggle/working/drone_data/train/images', exist_ok=True)
os.makedirs('/kaggle/working/drone_data/train/labels', exist_ok=True)

for img in os.listdir(IMAGE_PATH):
    if img.endswith('.jpg'):
        # Using copy instead of symlink for higher reliability in 'Save & Run All'
        shutil.copy(os.path.join(IMAGE_PATH, img), '/kaggle/working/drone_data/train/images/')

for lbl in os.listdir(LABEL_DIR):
    shutil.copy(os.path.join(LABEL_DIR, lbl), '/kaggle/working/drone_data/train/labels/')

# 2. Create YAML
data_yaml = {
    'path': '/kaggle/working/drone_data',
    'train': 'train/images',
    'val': 'train/images', 
    'nc': 2,
    'names': ['Plantable', 'Non-Plantable']
}

with open('/kaggle/working/drone_terrain.yaml', 'w') as f:
    yaml.dump(data_yaml, f)

# 3. Train
model = YOLO('yolov8n-seg.pt') 
model.train(
    data='/kaggle/working/drone_terrain.yaml',
    epochs=15, # Faster turnaround
    imgsz=640,
    device=0,
    project='Rover_Project',
    name='Plantable_Final'
)

# 4. ZIP EVERYTHING AUTOMATICALLY
# This ensures even if you can't see the folders, the ZIP is waiting for you
shutil.make_archive('Final_Project_Weights', 'zip', '/kaggle/working/runs/segment/Rover_Project/Plantable_Final')
print("🏁 COMPLETE. Download 'Final_Project_Weights.zip' from the Output tab!")

Ultralytics 8.4.16 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 14913MiB)
engine/trainer: agnostic_nms=False, amp=True, angle=1.0, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/kaggle/working/drone_terrain.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, end2end=None, epochs=15, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n-seg.pt, momentum=0.937, mosaic=1.0, multi_scale=0.0, name=Plantable_Final, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=1